# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
A. Hughes  ->  A. Hughes  |  ['A. Hughes']
T. K. Henning  ->  T. K. Henning  |  ['T. K. Henning']
F. Walter  ->  F. Walter  |  ['F. Walter']
Arxiv has 51 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2309.08078


extracting tarball to tmp_2309.08078...

 done.
Retrieving document from  https://arxiv.org/e-print/2309.08403


extracting tarball to tmp_2309.08403...

...

/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2309.08403/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2309.08515...

 done.


T. K. Henning  ->  T. K. Henning  |  ['T. K. Henning']
F. Walter  ->  F. Walter  |  ['F. Walter']


Found 116 bibliographic references in tmp_2309.08515/paper.bbl.
syntax error in line 634: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.08515-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.08515) | **MIDIS. JWST NIRCam and MIRI unveil the stellar population properties of  Ly$α$-emitters and Lyman-Break galaxies at z ~ 3-7**  |
|| E. Iani, et al. -- incl., <mark>T. K. Henning</mark>, <mark>F. Walter</mark> |
|*Appeared on*| *2023-09-18*|
|*Comments*| *19 pages, 15 figures, 5 tables. Submitted to APJ*|
|**Abstract**| We study the stellar population properties of 182 spectroscopically-confirmed (MUSE/VLT) Lyman-$\alpha$ emitters (LAEs) and 450 photometrically-selected Lyman-Break galaxies (LBGs) at z = 2.8 - 6.7 in the Hubble eXtreme Deep Field (XDF). Leveraging the combined power of HST and JWST NIRCam and MIRI observations, we analyse their rest-frame UV-through-near-IR spectral energy distributions (SEDs) with MIRI playing a crucial role in robustly assessing the LAE's stellar mass and ages. Our LAEs are low-mass objects (log$_{10}$(M$_\star$[M$_\odot$]) ~ 7.5), with little or no dust extinction (E(B - V) ~ 0.1) and a blue UV continuum slope ($\beta$ ~ -2.2). While 75% of our LAEs are young (< 100 Myr), the remaining 25% have significantly older stellar populations (> 100 Myr). These old LAEs are statistically more massive, less extinct and have lower specific star formation rate (sSFR) compared to young LAEs. Besides, they populate the M$_\star$ - SFR plane along the main-sequence (MS) of star-forming galaxies, while young LAEs populate the starburst region. The comparison between the LAEs properties to those of a stellar-mass matched sample of LBGs shows no statistical difference between these objects, except for the LBGs redder UV continuum slope and marginally larger E(B - V) values. Interestingly, 48% of the LBGs have ages < 10 Myr and are classified as starbursts, but lack detectable Ly$\alpha$ emission. This is likely due to HI resonant scattering and/or selective dust extinction. Overall, we find that JWST observations are crucial in determining the properties of LAEs and shedding light on the properties and similarities between LAEs and LBGs. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.08078-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.08078) | **A sub-pc BBH system in SDSS J1609+1756 through optical QPOs in ZTF light  curves**  |
|| <mark>X. Zhang</mark> |
|*Appeared on*| *2023-09-18*|
|*Comments*| *13 pages, 2 tables, 1 figures, Accepted to be published in MNRAS*|
|**Abstract**| Optical quasi-periodic oscillations (QPOs) are the most preferred signs of sub-pc binary black hole (BBH) systems in AGN. In this manuscript, robust optical QPOs are reported in quasar SDSS J1609+1756 at $z=0.347$. In order to detect reliable optical QPOs, four different methods are applied to analyze the 4.45 years-long ZTF g/r/i-band light curves of SDSS J1609+1756, direct fitting results by sine function, Generalized Lomb-Scargle periodogram, Auto-Cross Correlation Function and Weighted Wavelet Z-transform method. The Four different methods can lead to well determined reliable optical QPOs with periodicities $\sim340$ days with confidence levels higher than 5$\sigma$, to guarantee the robustness of the optical QPOs in SDSS J1609+1756. Meanwhile, based on simulated light curves through CAR process to trace intrinsic AGN activities, confidence level higher than $3\sigma$ can be confirmed that the optical QPOs are not mis-detected in intrinsic AGN activities, re-confirming the robust optical QPOs and strongly indicating a central sub-pc BBH system in SDSS J1609+1756. Furthermore, based on apparent red-shifted shoulders in broad Balmer emission lines in SDSS J1609+1756, space separation of the expected central BBH system can be estimated to be smaller than $107\pm60$ light-days, accepted upper limit of total BH mass $\sim(1.03\pm0.22)\times10^8{\rm M_\odot}$. Therefore, to detect and report BBH system expected optical QPOs with periodicities around 1 year is efficiently practicable through ZTF light curves, and combining with peculiar broad line emission features, further clues should be given on space separations of BBH systems in broad line AGN in the near future. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2309.08403-b31b1b.svg)](https://arxiv.org/abs/arXiv:2309.08403) | **The stratification of ISM properties in the edge-on galaxy NGC 891  revealed by NIKA2**  |
|| S. Katsioli, et al. -- incl., <mark>A. Hughes</mark> |
|*Appeared on*| *2023-09-18*|
|*Comments*| *15 pages, 10 figures, Accepted for publication in A&A*|
|**Abstract**| As the millimeter wavelength range remains a largely unexplored spectral region for galaxies, the IMEGIN large program aims to map the millimeter continuum emission of 22 nearby galaxies at 1.15 and 2 mm. Using the high-resolution maps produced by the NIKA2 camera, we explore the existence of very cold dust and take possible contamination by free-free and synchrotron emission into account. We study the IR-to-radio emission coming from different regions along the galactic plane and at large vertical distances. New observations of NGC 891, using the NIKA2 camera on the IRAM 30m telescope, along with a suite of observations at other wavelengths were used to perform a multiwavelength study of the spectral energy distribution in the interstellar medium in this galaxy. This analysis was performed globally and locally, using the advanced hierarchical Bayesian fitting code, HerBIE, coupled with the THEMIS dust model. Our dust modeling is able to reproduce the near-IR to millimeter emission of NGC 891, with the exception of an excess at a level of 25% obtained by the NIKA2 observations in the outermost parts of the disk. The radio continuum and thermal dust emission are distributed differently in the disk and galaxy halo. Different dusty environments are also revealed by a multiwavelength investigation of the emission features. Our detailed decomposition at millimeter and centimeter wavelengths shows that emission at 1 mm is purely originated by dust. Radio components become progressively important with increasing wavelengths. Finally, we find that emission arising from small dust grains accounts for ~ 9.5% of the total dust mass, reaching up to 20% at large galactic latitudes. Shock waves in the outflows that shatter the dust grains might explain this higher fraction of small grains in the halo. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2309.08515.md
    + _build/html/tmp_2309.08515/./figures/ebv_dist.png
    + _build/html/tmp_2309.08515/./figures/mass_dist.png
    + _build/html/tmp_2309.08515/./figures/age_dist.png
    + _build/html/tmp_2309.08515/./figures/beta_dist.png
    + _build/html/tmp_2309.08515/./figures/muv_dist.png
    + _build/html/tmp_2309.08515/./figures/sfr_dist.png
    + _build/html/tmp_2309.08515/./figures/ssfr_dist.png
    + _build/html/tmp_2309.08515/./figures/AGE_BEST_massmatch.png
    + _build/html/tmp_2309.08515/./figures/EBV_BEST_massmatch.png
    + _build/html/tmp_2309.08515/./figures/BETA_SLOPE_massmatch.png
    + _build/html/tmp_2309.08515/./figures/MUV_corr_massmatch.png
    + _build/html/tmp_2309.08515/./figures/SFR_UV_massmatch.png
    + _build/html/tmp_2309.08515/./figures/sSFR_UV_massmatch.png
    + _build/html/tmp_2309.08515/./figures/llya_z.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\jwst}{{\it JWST}}$
$\newcommand{\hst}{{\it HST}}$
$\newcommand{\lya}{Ly\alpha}$
$\newcommand{\lephare}{\textsc{LePHARE}}$
$\newcommand{\ebv}{E(B-V)}$
$\newcommand{\mstar}{M_\star}$
$\newcommand{\python}{\textsc{Python}}$</div>



<div id="title">

# MIDIS. $\jwst$ NIRCam and MIRI unveil the stellar population properties of $\lya$-emitters and Lyman-Break galaxies at $z \simeq 3 - 7$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2309.08515-b31b1b.svg)](https://arxiv.org/abs/2309.08515)<mark>Appeared on: 2023-09-18</mark> -  _19 pages, 15 figures, 5 tables. Submitted to APJ_

</div>
<div id="authors">

E. Iani, et al. -- incl., <mark>T. K. Henning</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** We study the stellar population properties of 182 spectroscopically-confirmed (MUSE/VLT) Lyman- $\alpha$ emitters (LAEs) and 450 photometrically-selected Lyman-Break galaxies (LBGs) at $z = 2.8 - 6.7$ in the Hubble eXtreme Deep Field (XDF).Leveraging the combined power of $\hst$ and $\jwst$ NIRCam and MIRI observations, we analyse their rest-frame UV-through-near-IR spectral energy distributions (SEDs) with MIRI playing a crucial role in robustly assessing the LAE's stellar mass and ages.Our LAEs are low-mass objects ( $\rm \log_{10}(M_\star [M_\odot]) \simeq 7.5$ ), with little or no dust extinction ( $\rm E(B-V) \simeq 0.1$ ) and a blue UV continuum slope ( $\beta \simeq -2.2$ ).While 75 \% of our LAEs are young ( $<$ 100 Myr), the remaining 25 \% have significantly older stellar populations ( $\geq 100$ Myr).These old LAEs are statistically more massive, less extinct and have lower specific star formation rate (sSFR) compared to young LAEs. Besides, they populate the M $_\star$ -- SFR plane along the main-sequence (MS) of star-forming galaxies, while young LAEs populate the starburst region.The comparison between the LAEs properties to those of a stellar-mass matched sample of LBGs shows no statistical difference between these objects, except for the LBGs redder UV continuum slope and marginally larger E(B-V) values.Interestingly, 48 \% of the LBGs have ages $<$ 10 Myr and are classified as starbursts, but lack detectable $\lya$ emission. This is likely due to HI resonant scattering and/or selective dust extinction.Overall, we find that $\jwst$ observations are crucial in determining the properties of LAEs and shedding light on the properties and similarities between LAEs and LBGs.

</div>

<div id="div_fig1">

<img src="tmp_2309.08515/./figures/ebv_dist.png" alt="Fig11.1" width="14%"/><img src="tmp_2309.08515/./figures/mass_dist.png" alt="Fig11.2" width="14%"/><img src="tmp_2309.08515/./figures/age_dist.png" alt="Fig11.3" width="14%"/><img src="tmp_2309.08515/./figures/beta_dist.png" alt="Fig11.4" width="14%"/><img src="tmp_2309.08515/./figures/muv_dist.png" alt="Fig11.5" width="14%"/><img src="tmp_2309.08515/./figures/sfr_dist.png" alt="Fig11.6" width="14%"/><img src="tmp_2309.08515/./figures/ssfr_dist.png" alt="Fig11.7" width="14%"/>

**Figure 11. -** Distributions of the physical properties for our sample of LAEs (in blue) and LBGs (in green). In the top row we present the colour excess, stellar mass and age as derived from the $\lephare$ SED fitting, while in the central and bottom rows, we display the distribution of the parameters directly derived from the analysis of the UV photometry of our sources, i.e. the UV continuum slope ($\beta$), the UV absolute magnitude, the star formation rate and the specific star formation rate. (*fig:sed_prop_hist*)

</div>
<div id="div_fig2">

<img src="tmp_2309.08515/./figures/AGE_BEST_massmatch.png" alt="Fig15.1" width="16%"/><img src="tmp_2309.08515/./figures/EBV_BEST_massmatch.png" alt="Fig15.2" width="16%"/><img src="tmp_2309.08515/./figures/BETA_SLOPE_massmatch.png" alt="Fig15.3" width="16%"/><img src="tmp_2309.08515/./figures/MUV_corr_massmatch.png" alt="Fig15.4" width="16%"/><img src="tmp_2309.08515/./figures/SFR_UV_massmatch.png" alt="Fig15.5" width="16%"/><img src="tmp_2309.08515/./figures/sSFR_UV_massmatch.png" alt="Fig15.6" width="16%"/>

**Figure 15. -** Comparison between the distribution of the physical properties of LAEs (in blue) and mass-matched LBGs (in green). The vertical solid lines highlight the median value of the distributions, while the dashed lines are indicative of the 68\% confidence interval, i.e. the 16th and 84th percentiles, respectively. (*fig:mass_matched*)

</div>
<div id="div_fig3">

<img src="tmp_2309.08515/./figures/llya_z.png" alt="Fig1" width="100%"/>

**Figure 1. -** Observed $\lya$ luminosity vs. redshift diagram of our final sample of LAEs. We highlight with orange and red edges LAEs matched to two and three photometric counterparts, respectively.
    The top and right panels show the redshift and $\lya$ luminosity distribution of our source. The grey shaded region indicates the luminosity corresponding to the 2$\sigma$ depth of the MUSE observations \citep{Bacon+23}, i.e. $2.1 \times 10^{-19}$ and $4.2 \times 10^{-20}$ erg/s/cm$^2$ at 10- and 141-h depths, respectively. (*fig:z-llya*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2309.08515"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

271  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
